# Sequential API

Sequential API реализуется посредством класса `nn.Sequential` и подходит для простых линейных архитектур. В конструкторе слоям присваиваются имена (`0, 1, ...`). Всякий компонент сети, который можно состыковать с другими компонентами, будь это слой или целая сеть, называется модулем.

> Для явного именования модулей в конструкторе `nn.Sequential` можно использовать `OrderedDict`.

In [ ]:
from torch import nn 

flatten = nn.Flatten()  # создается слой для выравнивания тензора

seq_modules = nn.Sequential(
    flatten,    # добавление ранее созданного модуля
    nn.Linear(in_features=28*28, out_features=20),
    nn.ReLU(),
    nn.Linear(20, 10),
)

# добавление модуля после того, как последовательность создана
seq_modules.add_module(name="softmax", module=nn.Softmax())

seq_modules

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=20, bias=True)
  (2): ReLU()
  (3): Linear(in_features=20, out_features=10, bias=True)
  (softmax): Softmax(dim=None)
)

При добавлении модуля вне конструктора класса при помощи метода `add_module()` необходимо задавать значение `name`. Если при этом заданное значение уже существует в модели, то этот модуль будет заменен добавляемым модулем.

Доступ к модулю можно получить по индексу:

In [42]:
seq_modules[1]

Linear(in_features=784, out_features=20, bias=True)

Доступ к модулю можно получить также и по имени слоя, если обратиться к полю `_modules`, в котором содержатся слои в виде словаря, ключами которого являются имена:

In [43]:
seq_modules._modules["softmax"]

Softmax(dim=None)

# Subclassing API

Для полного контроля над архитектурой необходимо создать класс, наследующий класс `Module` из модуля `nn`. Модули определяются в конструкторе, а в методе `forward` задается специфика прохождения данных через сеть. Напрямую этот метод вызывать не сто́ит. Полный API [здесь](https://pytorch.org/docs/stable/nn.html).

Так как сеть, построенная при помощи `nn.Sequential()` является полноценным модулем, то ничего не мешает создать такой (или использовать готовый) модуль в конструкторе.

In [5]:
from torch import nn

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(in_features=28*28,
                      out_features=512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
        self.add_module("softmax", nn.Softmax())

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = NeuralNetwork()
model

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
  (softmax): Softmax(dim=None)
)

Заметим, что имена модулям присваиваются в соответствии с именами полей. При желании можно определить непосредственно имя модуля, если добавлять его при помощи метода `add_module()`.

Доступ к модулям может быть произведен как через непосредственное обращение к полю, так и через имя:

In [10]:
model.flatten, model._modules["softmax"]

(Flatten(start_dim=1, end_dim=-1), Softmax(dim=None))

Результат возвращаемый нейронным слоем без функции активации принято называть `logits` (не путать с функцией logit!), и содержит произвольные значения. В классе `NeuralNetwork` мы добавилди модуль `softmax` в конструкторе, но не задействован в методе `forward()`. 

Для ускорения операций мы можем задать специфику при создании объекта модели. Для начала выясним, что есть в доступе:

In [ ]:
device = (
         "cuda" if torch.cuda.is_available()
    # else "mps"  if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

In [ ]:
model = NeuralNetwork().to(device)
print(model)

Для прогноза можно вызывать модель, передав ей данные. Модель возвратит двумерный тензор, в котором первая размерность (`dim=0`) соответствует образцам, а вторая размерность (`dim=1`) соответствует выходным значениям сети.

Мы можем применить отдельно слой `Softmax`, чтобы получить распределение вероятностей (в конструкторе указывается измерение, вдоль которого применяется softmax):

In [ ]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
print(pred_probab)

In [ ]:
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {int(y_pred)}")

В выражении `nn.Softmax(dim=1)(logits)` создается объект слоя `Softmax` и тут же вызывается с входным значением `logits`. Таким образом мы вольны создавать отдельные слои, или последовательность слоев, не определяя для этого пользовательский класс. Для этого используется упорядоченный контейнер для модулей `Sequential`. Данные пройдут через все модули в том же порядке, как и определены.

In [ ]:
flatten = nn.Flatten()

seq_modules = nn.Sequential(
    flatten,    # добавление ранее созданного модуля
    nn.Linear(in_features=28*28, out_features=20),
    nn.ReLU(),
    nn.Linear(20, 10),
)
# добавление модуля после того, как последовательность создана
seq_modules.add_module(name="4", module=nn.Softmax())
seq_modules

# Training

Определим [оптимизатор](https://pytorch.org/docs/stable/optim.html) и [функцию потерь](https://pytorch.org/docs/stable/nn.html#loss-functions). Заметим, что первым аргументом в конструктор оптимизатора передается генератор, содержащий параметры модели.

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

Определим функцию для одной эпохи. Модель производит прогнозы для тренировочного набора, скармливая данные партиями, и производит обратное распространение ошибки для коррекции параметров. Оптимизация производится в три этапа:

- Вызывается метод `zero_grad()` оптимизатора, для того, чтобы сбросить градиенты параметров модели, так как по умолчанию, градиенты будут складываться.
- Вызывается метод `backward()` у целевой функции, чтобы произвести обратное распространение. Вычисляемые при этом градиенты сохраняются в тензорах, в которых хранятся параметры.
- После того, как градиенты получены, вызывается метод `step()` оптимизатора, чтобы произвести коррекцию параметров.

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

Для того, чтобы отслеживать работу сети на тестовых данных, создадим тестовую функцию:

In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

Запустим обучение на 5 эпохах. Будем выводить accuracy и loss на каждой эпохе:

In [ ]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

# Saving and Loading Models

Функция `save()` позволяет сохранить модель путем сериализации внутреннего словаря состояний, содержащего параметры модели:


In [ ]:
torch.save(model.state_dict(), "model.pth")

Для загрузки необходимо создать модель соответствующей структуры, и загрузить в него словарь состояний. Для загрузки словаря используется функция `load()`, а для того, чтобы установить эти данные в модели используется методы `load_state_dict()`.

In [ ]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

Более подробно о сохранении и загрузке моделей [здесь](https://pytorch.org/tutorials/beginner/basics/saveloadrun_tutorial.html).

# Inference

Для перевода модели в режим инференса, необходимо вызвать метод `eval()`. Это оптимизирует вычисление: отключаются dropout и batch normalization, не вычисляются градиенты и тд.

In [ ]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

# model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')